# array-oriented design

1. Design interface with arrays
2. Conversion between dynamic and static semantics
3. Insert profiling code

# Design interface with arrays

There are multiple ways to access arrays in C++, from Python.  But no matter what approach is used, we must be careful about the object (memory) ownership.  There are some guidelines:

* Use arrays when the data need to go to Python.
* Create ndarray from a pointer to a buffer.
* C++ constness.  Python doesn't have the concept of constness (Python has mutability).
  * Ndarray is mutable.  Returning a buffer as ndarray from a const C++ object violates the constness and it may make the data inconsistent.
  * Returning a copy of the buffer preserves the constness, but incurs overhead.
* Prefer structure of arrays to array of structure.

## Struct of array and array of struct

```cpp
struct StructOfArray
{
    std::vector<double> x;
    std::vector<double> y;
};

struct PointProxy
{
    StructOfArray * soa;
    size_t idx;
    double   x() const { return soa.x[idx]; }
    double & x()       { return soa.x[idx]; }
    double   y() const { return soa.y[idx]; }
    double & y()       { return soa.y[idx]; }
};

/*
 * Array of struct:
 */
struct Point
{
    double x, y;
};

using ArrayOfStruct = std::vector<Point>;
```

# Conversion between dynamic and static semantics

## Do it your self

It's not a bad idea to do it manually.  Spelling the static to dynamic conversion makes it clear what do we want to do.  When we work in the inner-most loop, no `PyObject` or virtual function table should be there.

```cpp
template <size_t ND>
class SpaceBase
{
public:
    static constexpr const size_t NDIM = ND;
    using serial_type = uint32_t;
    using real_type = double;
}; /* end class SpaceBase */

class StaticGrid1d
  : public StaticGridBase<1>
{
}; /* end class StaticGrid1d */

class StaticGrid2d
  : public StaticGridBase<2>
{
}; /* end class StaticGrid2d */

class StaticGrid3d
  : public StaticGridBase<3>
{
}; /* end class StaticGrid3d */

/*
 * WrapStaticGridBase has the pybind11 wrapping code.
 */

class WrapStaticGrid1d
  : public WrapStaticGridBase< WrapStaticGrid1d, StaticGrid1d >
{
}; /* end class WrapStaticGrid1d */

class WrapStaticGrid2d
  : public WrapStaticGridBase< WrapStaticGrid2d, StaticGrid2d >
{
}; /* end class WrapStaticGrid2d */

class WrapStaticGrid3d
  : public WrapStaticGridBase< WrapStaticGrid3d, StaticGrid3d >
{
}; /* end class WrapStaticGrid3d */

PYBIND11_MODULE(_modmesh, mod)
{
    WrapStaticGrid1d::commit(mod);
    WrapStaticGrid2d::commit(mod);
    WrapStaticGrid3d::commit(mod);
}
```

## Example: `pybind11::cppfunction`

### `pybind11::cppfunction`

https://github.com/pybind/pybind11/blob/v2.4.3/include/pybind11/pybind11.h#L56

```cpp
/// Wraps an arbitrary C++ function/method/lambda function/.. into a callable Python object
class cpp_function : public function {
public:
    cpp_function() { }
    cpp_function(std::nullptr_t) { }

    /// Construct a cpp_function from a vanilla function pointer
    template <typename Return, typename... Args, typename... Extra>
    cpp_function(Return (*f)(Args...), const Extra&... extra) {
        initialize(f, f, extra...);
    }

    /// Construct a cpp_function from a lambda function (possibly with internal state)
    template <typename Func, typename... Extra,
              typename = detail::enable_if_t<detail::is_lambda<Func>::value>>
    cpp_function(Func &&f, const Extra&... extra) {
        initialize(std::forward<Func>(f),
                   (detail::function_signature_t<Func> *) nullptr, extra...);
    }

    /// Construct a cpp_function from a class method (non-const)
    template <typename Return, typename Class, typename... Arg, typename... Extra>
    cpp_function(Return (Class::*f)(Arg...), const Extra&... extra) {
        initialize([f](Class *c, Arg... args) -> Return { return (c->*f)(args...); },
                   (Return (*) (Class *, Arg...)) nullptr, extra...);
    }

    /// Construct a cpp_function from a class method (const)
    template <typename Return, typename Class, typename... Arg, typename... Extra>
    cpp_function(Return (Class::*f)(Arg...) const, const Extra&... extra) {
        initialize([f](const Class *c, Arg... args) -> Return { return (c->*f)(args...); },
                   (Return (*)(const Class *, Arg ...)) nullptr, extra...);
    }

// ...
```

### `pybind11::cppfunction::initialize`

https://github.com/pybind/pybind11/blob/v2.4.3/include/pybind11/pybind11.h#L98

```cpp
/// Special internal constructor for functors, lambda functions, etc.
template <typename Func, typename Return, typename... Args, typename... Extra>
void initialize(Func &&f, Return (*)(Args...), const Extra&... extra) {
// ...
```

### `pybind11::cppfunction::dispatch`

https://github.com/pybind/pybind11/blob/v2.4.3/include/pybind11/pybind11.h#L423

```cpp
static PyObject *dispatcher(PyObject *self, PyObject *args_in, PyObject *kwargs_in) {
// ...
```

# Insert profiling code

In addition to using OS-provided profiling tools, e.g., Linux's perf and Macos's Instruments, we should also add a custom profiling layer in the code.  You may need to port your code to a platform that doens't have very good system profiler.  Your custom profiler will become the safety net.

```cpp
/*
 * MODMESH_PROFILE defined: Enable profiling API.
 */
#ifdef MODMESH_PROFILE

#define MODMESH_TIME(NAME) \
    ScopedTimer local_scoped_timer_ ## __LINE__(NAME);

/*
 * No MODMESH_PROFILE defined: Disable profiling API.
 */
#else // MODMESH_PROFILE

#define MODMESH_TIME(NAME)

#endif // MODMESH_PROFILE
/*
 * End MODMESH_PROFILE.
 */

struct ScopedTimer
{

    ScopedTimer() = delete;

    ScopedTimer(const char * name) : m_name(name) {}

    ~ScopedTimer()
    {
        TimeRegistry::me().add(m_name, m_sw.lap());
    }

    StopWatch m_sw;
    char const * m_name;

}; /* end struct ScopedTimer */

// Manually
void StaticGrid1d::fill(StaticGrid1d::real_type val)
{
    MODMESH_TIME("StaticGrid1d::fill");
    std::fill(m_coord.get(), m_coord.get()+m_nx, val);
}
```

## Profiler turned off

In [ ]:
!rm -rf modmesh/build ; make -C modmesh buildext

In [ ]:
!./fill.py

## Profiler turned on

In [ ]:
!touch modmesh/Makefile ; make -C modmesh buildext MODMESH_PROFILE=ON

In [ ]:
!./fill.py

# References

1. https://github.com/yse/easy_profiler
2. http://www.brofiler.com